<a href="https://colab.research.google.com/github/matheussantosguedes00/Imersao-Inteligencia-Artificial-Alura/blob/main/City_Hotel_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# **City Hotel Assistant**

 **Descrição do Chatbot City Hotel Assistant:**
O City Hotel Assistant é um chatbot com o objetivo de auxiliar usuários na busca por hotéis em uma cidade específica. Sua interface é amigável e fácil de usar. No entanto, é importante ter em mente que os resultados podem não ser totalmente precisos, já que o banco de dados utilizado pode não estar completamente atualizado.

**Funcionalidades:**

*   Pesquisa de hotéis por cidade.
*   Possibilidade de filtrar resultados por critérios como preço, localização e avaliações.
*   Fornecimento de informações básicas sobre os hotéis, como endereço, telefone e site

**Limitações:**

*   Precisão dos resultados pode ser afetada por um banco de dados desatualiza
*   Pode não incluir todos os hotéis disponíveis na cidade.


**Público-alvo:**

Viajantes que buscam uma maneira rápida e fácil de encontrar hotéis em uma cidade específica.

**Conclusão:**

O City Hotel Assistant pode ser uma ferramenta útil para quem procura hotéis, mas é importante estar ciente de suas limitações e utilizar outras fontes para confirmar as informações antes de tomar uma decisão.


**Instalando o SDK do Google**

In [1]:
!pip install -q -U google-generativeai

**Colocando api key do GOOGLE**

In [2]:
import google.generativeai as genai


GOOGLE_API_KEY="adicionar API Key  "
genai.configure(api_key=GOOGLE_API_KEY)

**Listar os modelos disponiveis**

In [3]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


**Isso no caso seria a configuração da Api**

* Candidate_count -> isso e usado para não da mais de uma resposta

* Temperature -> e usado para colocar aleatoriedade no modelo nas escolhas das palavras, Quando estiver maior mais criativo vai sair um pouco do modelo conservador.

In [4]:
generation_config = {
  "candidate_count": 1,
  "temperature": 0.5,
}

**Usado para colocar segurança na escrita e não deixa colocar palavra inapropriadas**

In [5]:
safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }

**Inicializando o modelo**

In [6]:
model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                                  generation_config=generation_config,
                                  safety_settings=safety_settings,)

**Função para colocar dados na tabela**

In [25]:
def formatar_tabela(tabela, separador='|'):
    """Formata uma tabela de texto utilizando caracteres especiais."""
    linhas = tabela.strip().split('\n')

    if len(linhas) < 2:
        raise ValueError("A tabela deve ter pelo menos um cabeçalho e uma linha de dados.")

    cabecalho, corpo = extrair_cabecalho_e_corpo(linhas)
    colunas_tamanho = determinar_tamanho_colunas(cabecalho, corpo)

    cabecalho_formatado = formatar_linha(cabecalho, colunas_tamanho, separador)
    separador_formatado = formatar_separador(colunas_tamanho, separador)
    corpo_formatado = formatar_corpo(corpo, colunas_tamanho, separador)

    tabela_formatada = f"{separador_formatado}\n{cabecalho_formatado}\n{separador_formatado}\n{corpo_formatado}{separador_formatado}"

    return tabela_formatada

def extrair_cabecalho_e_corpo(linhas):
    """Extrai o cabeçalho e o corpo da tabela."""
    cabecalho = linhas[0].split(';')
    corpo = [linha.split(';') for linha in linhas[1:]]
    return cabecalho, corpo

def determinar_tamanho_colunas(cabecalho, corpo):
    """Determina o tamanho das colunas com base no conteúdo."""
    colunas_tamanho = [max(len(str(item)) for item in coluna) for coluna in zip(cabecalho, *corpo)]
    return colunas_tamanho

def formatar_linha(linha, colunas_tamanho, separador):
    """Formata uma linha da tabela."""
    return separador + separador.join(
        f"{item.center(tamanho)} " for item, tamanho in zip(linha, colunas_tamanho)
    ) + separador

def formatar_separador(colunas_tamanho, separador):
    """Formata a linha de separação."""
    return f"+{'+'.join(['-' * (tamanho + 2) for tamanho in colunas_tamanho])}+"

def formatar_corpo(corpo, colunas_tamanho, separador):
    """Formata o corpo da tabela."""
    corpo_formatado = ""
    for linha in corpo:
        corpo_formatado += separador + separador.join(
            f"{item.ljust(tamanho)} " for item, tamanho in zip(linha, colunas_tamanho)
        ) + separador + '\n'
        corpo_formatado += '\n'  # Adiciona uma quebra de linha após cada linha da tabela
    return corpo_formatado

# Exemplo de uso (substitua pelo seu modelo)
def obter_hoteis(cidade, categoria):
    """Simulação de obtenção de dados de hotéis."""
    # Substitua esta parte pela chamada ao seu modelo
    # O retorno deve ser uma string no formato de tabela
    return f"Nome;Endereço;Valor\nHotel A;Rua X, 123;R$ 100\nHotel B;Avenida Y, 456;R$ 200"

chat = model.start_chat(history=[])  # Substitua 'model' pelo seu modelo
prompt = input('Digite a cidade: ')
prompt2 = input('Hotéis de Categoria (Econômicos, Média, Luxo): ')

while prompt.lower() != "fim" or prompt2.lower() != "fim":
    try:
        response = chat.send_message("Hoteis em " + prompt + " Categoria " + prompt2 + " Retorna em uma tabela mais bonita com Nome;Endereço;Nota;Vantagem")
        tabela_formatada = formatar_tabela(response.text)
        print("Resposta:\n", tabela_formatada, '\n\n')
    except ValueError as e:
        print(f"Erro: {e}")

    prompt = input('Digite a cidade ou "fim" para encerrar: ')
    if prompt.lower() == "fim":
        break  # Sai do loop se "fim" for digitado em 'prompt'
    prompt2 = input('Hotéis de Categoria (Econômicos, Média, Luxo): ')
    if prompt2.lower() == "fim":
        break  # Sai do loop se "fim" for digitado em 'prompt2'


Digite a cidade: passos
Hotéis de Categoria (Econômicos, Média, Luxo): luxo
Resposta:
 +--------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                     | Nome | Endereço | Nota | Vantagem |                                                       |
+--------------------------------------------------------------------------------------------------------------------------------------------------+
||---|---|---|---|                                                                                                                                |

|| Hotel Fazenda Vale das Pedras | Rodovia MG-050, KM 108,5 | 9,5 | Área de lazer completa com piscinas, quadras esportivas e passeios a cavalo | |

|| Pousada Villa do Império | Rua Tiradentes, 150 | 9,0 | Charme e aconchego em um casarão histórico no centro da cidade |                        |

|| Hotel Fazenda Rec